# GISTDA Wildfire Machine Learning Training

## Import and Read SHAPE File

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import rasterio
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import display, Markdown
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from dask import delayed, compute
from rasterio.windows import Window

#pd.set_option("display.max_columns", None)  # To show all columns in a pandas DataFrame

# Define the folder containing the raster files
raster_train_file_path = r'Raster\output'

# Parameters for chunk size
CHUNK_SIZE = 1024

@delayed
def read_raster_in_chunks(raster_path, file, root):
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        num_bands = src.count
        band_names = [f'B{str(i).zfill(2)}' for i in range(1, num_bands + 1)]
        chunk_dfs = []
        
        # Loop over the raster in chunks
        for row in range(0, height, CHUNK_SIZE):
            for col in range(0, width, CHUNK_SIZE):
                window = Window(col_off=col, row_off=row, 
                              width=min(CHUNK_SIZE, width - col),
                              height=min(CHUNK_SIZE, height - row))
                
                # Read all bands at once
                data = src.read(window=window)
                
                # Check if chunk contains any data
                if np.any(data):
                    rows, cols = data[0].shape
                    
                    # Create base DataFrame with coordinates
                    row_coords, col_coords = np.meshgrid(
                        np.arange(row, row + rows),
                        np.arange(col, col + cols),
                        indexing="ij"
                    )
                    
                    chunk_df = pd.DataFrame({
                        'raster_file': file,
                        'subfolder': os.path.basename(root),
                        'x': row_coords.flatten(),
                        'y': col_coords.flatten()
                    })
                    
                    # Add each band's data
                    for band_idx, band_name in enumerate(band_names, 1):
                        chunk_df[band_name] = data[band_idx-1].flatten()
                    
                    chunk_dfs.append(chunk_df)
        
        return pd.concat(chunk_dfs, ignore_index=True) if chunk_dfs else pd.DataFrame()

# Create list of tasks
dask_dfs = [
    read_raster_in_chunks(os.path.join(root, file), file, root)
    for root, dirs, files in os.walk(raster_train_file_path)
    for file in files if file.endswith('.tif')
]

# Compute all tasks
dataframes = compute(*dask_dfs)

# Combine all DataFrames
final_df = pd.concat(dataframes, ignore_index=True)

# Debug prints
print("DataFrame shape:", final_df.shape)
print("\nDataFrame columns:", final_df.columns.tolist())
print("\nSample of data:")
print(final_df.head())

## Exploratory Data Analysis (EDA) & Feature Engineering

In [ ]:
# Convert pandas DataFrame to dask DataFrame if already loaded
ddf = pd.DataFrame(final_df)  # Adjust number of partitions as needed

## Rename Sentinel-2 Bands columns and Burn Label
# List of new column names
new_col_names = ['raster_file', 'subfolder', 'x', 'y', 'Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 
                 'Band_6', 'Band_7', 'Band_8', 'Band_8A', 'Band_9', 'Band_11', 'Band_12', 'dNBR',
                 'NDVI', 'NDWI', 'Burn_Label']


# Renaming columns using the list
ddf.columns = new_col_names

# Drop columns in dask
df = ddf.drop(columns=['raster_file', 'subfolder', 'x', 'y', 'dNBR'])
display(df)  # Compute only when you need to display or save results

## Check Burn Class

In [ ]:
# Check Burn Records
burn_counts = df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

### Downsampling

In [ ]:
burn_count = burn_counts['Burn']
unburn_sample = df[df['Burn_Label'] == 0].sample(n=burn_count, random_state=42)

downsampled_df = pd.concat([df[df['Burn_Label'] == 1], unburn_sample])

# Check Burn Records
burn_counts = downsampled_df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

## Pre-Processing

### Remove infinite values

In [ ]:
# Replacing infinite with nan 
downsampled_df.replace([np.inf, -np.inf], np.nan, inplace=True) 
  
# Dropping all the rows with nan values 
downsampled_df.dropna(inplace=True)

# Printing df 
display(downsampled_df)

### Seperate Burn_Label from DataFrame

In [ ]:
# Seperate Burn_Label from DataFrame
burn_label = downsampled_df[['Burn_Label']]

# Drop Label from DataFrame
downsampled_df = downsampled_df.drop(columns=['Burn_Label'])

# Change type of Label to Integer Format
burn_label = burn_label.astype('int32')
display(burn_label)

### Normalization Data with MinMax Scaler

In [ ]:
# Reassign the dataframe with a list of the columns
cols_norm = downsampled_df.columns.tolist()

# Import Normalize technique
scaler = MinMaxScaler()

# Normalize data
scaler.fit(downsampled_df)

# Save the scaler
scaler_save_path = r'Export Model'
save_path = os.path.join(scaler_save_path, 'MinMax_Scaler_DownSP.pkl')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, 'wb') as f:
    pickle.dump(scaler, f)

# Normalize Data
df_norm = scaler.transform(downsampled_df)
df_norm = pd.DataFrame(df_norm, columns=cols_norm)

# Check df_norm shape after normalization
print("Shape of df_norm after normalization:", df_norm.shape)

# Concatenate df_norm with burn_label
df_norm = pd.concat([df_norm.reset_index(drop=True), burn_label.reset_index(drop=True)], axis=1, sort=False)
display(df_norm)

## Machine Learning Model Training

### LightGBM Model Training

In [ ]:
def light_gbm(lgbm_learning_rate, num_leaves, df_norm):
    # Define the features and target
    X = df_norm.drop(columns=['Burn_Label'])
    Y = df_norm['Burn_Label']
    
    max_depth_range = range(2, 11)  # Range of max_depth to test (2 to 10)
    cv_scores = []  # Store mean cross-validation scores
    cv_std_devs = []  # Store standard deviation of cross-validation scores
    
    print("Performing hyperparameter tuning for max_depth in range 2-10...")
    
    for max_depth in max_depth_range:
        params = {
            'verbose': -1,
            'learning_rate': lgbm_learning_rate,
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc',
            'max_depth': max_depth,
            'num_leaves': num_leaves,
            'device': 'gpu'  # Use GPU for computation
        }
        
        # Initialize the LightGBM model
        lgbm_model = LGBMClassifier(**params)
        
        # 10-fold cross-validation
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        lgbm_scores = cross_val_score(lgbm_model, X, Y, cv=kf)
        
        # Store the mean and std of cross-validation scores
        cv_scores.append(lgbm_scores.mean())
        cv_std_devs.append(lgbm_scores.std())
        
        print(f"Max Depth: {max_depth}, CV Score: {round(lgbm_scores.mean() * 100, 2)}%, Std Dev: {round(lgbm_scores.std() * 100, 2)}%")
    
    # Find the optimal max_depth based on highest mean CV score
    optimal_index = np.argmax(cv_scores)
    optimal_max_depth = max_depth_range[optimal_index]
    best_cv_score = cv_scores[optimal_index]
    
    print(f"\nOptimal Max Depth: {optimal_max_depth} with CV Score: {round(best_cv_score * 100, 2)}%")
    
    # Plot the cross-validation results
    plt.figure(figsize=(10, 6))
    plt.plot(max_depth_range, cv_scores, marker='o', linestyle='-', color='b', label='Mean CV Score')
    plt.fill_between(max_depth_range, 
                     np.array(cv_scores) - np.array(cv_std_devs), 
                     np.array(cv_scores) + np.array(cv_std_devs), 
                     color='lightblue', alpha=0.5, label='Std Dev')
    plt.title('Hyperparameter Tuning: Max Depth vs CV Score')
    plt.xlabel('Max Depth')
    plt.ylabel('Mean CV Score')
    plt.xticks(max_depth_range)
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()
    
    # Fit the model with the optimal max_depth
    print(f"Fitting the final model with optimal max_depth = {optimal_max_depth}...")
    final_params = {
        'verbose': -1,
        'learning_rate': lgbm_learning_rate,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': optimal_max_depth,
        'num_leaves': num_leaves,
    }
    
    lgbm_model = LGBMClassifier(**final_params)
    lgbm_model.fit(X, Y)
    
    print("\nModel fitting complete.")
    
    # Generate classification report and confusion matrix
    print("\nGenerating classification report and confusion matrix...")
    y_pred = cross_val_predict(lgbm_model, X, Y, cv=kf)
    report = classification_report(Y, y_pred, output_dict=True)
    cm = confusion_matrix(Y, y_pred)
    
    # Create a summary of the results
    lgbm_result = [{
        'Classifier': 'LightGBM',
        'Model Definition': lgbm_model,
        'Class 0 - Precision': report['0']['precision'],
        'Class 0 - Recall': report['0']['recall'],
        'Class 0 - F1-Score': report['0']['f1-score'],
        'Class 1 - Precision': report['1']['precision'],
        'Class 1 - Recall': report['1']['recall'],
        'Class 1 - F1-Score': report['1']['f1-score'],
        'Average - Precision': report['macro avg']['precision'],
        'Average - Recall': report['macro avg']['recall'],
        'Average - F1-Score': report['macro avg']['f1-score'],
        'Accuracy': report['accuracy'],
        'Confusion Matrix': cm
    }]
    
    lgbm_result_df = pd.DataFrame(lgbm_result)
    
    display(Markdown("### Classification Report of LightGBM (Best Max Depth)"))
    display(lgbm_result_df)
    
    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix - LightGBM (Best Max Depth)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    return lgbm_model, optimal_max_depth, best_cv_score

# Call the function with the parameters and your DataFrame
lgbm_model, optimal_max_depth, best_cv_score = light_gbm(0.05, 31, df_norm)

print(f"Model trained with optimal max_depth: {optimal_max_depth}, achieving best CV score of: {round(best_cv_score * 100, 2)}%")

### Export LightGBM Model as pickle

In [ ]:
savepath = r'Export Model'
lgbm_filename_model = os.path.join(savepath, 'Model_LGBM_DownSP.sav')
pickle.dump(lgbm_model, open(lgbm_filename_model, 'wb'))

## Random Forest

In [ ]:
def random_forest(df, random_forest_estimator_num=100, criterion="gini"):
    """Performs hyperparameter tuning for Random Forest to find optimal max_depth
    and plots the results."""
    display(Markdown("### Hyperparameter Tuning for Random Forest"))
    print()  # Add Blank Line

    # Define the features and target
    X = df.drop(columns=['Burn_Label'])  # Features: all columns except Burn_Label
    Y = df['Burn_Label']                 # Target: Burn_Label column

    # Set up cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  # 10-fold cross-validation
    
    max_depths = list(range(2, 11))  # Range of max_depth to test
    mean_cv_scores = []

    # Perform hyperparameter tuning
    for max_depth in max_depths:
        rdf_model = RandomForestClassifier(n_estimators=random_forest_estimator_num,
                                           max_depth=max_depth,
                                           criterion=criterion,
                                           random_state=42)
        scores_cv = cross_val_score(rdf_model, X, Y, cv=kf)
        mean_cv_scores.append(scores_cv.mean())

    # Plotting the mean cross-validation accuracy vs. max_depth
    plt.figure(figsize=(10, 6))
    plt.plot(max_depths, mean_cv_scores, marker='o', linestyle='-', color='blue')
    plt.xlabel('Max Depth')
    plt.ylabel('Mean CV Accuracy')
    plt.title('Random Forest - Max Depth vs. Mean CV Accuracy')
    plt.xticks(max_depths)
    plt.grid(True)
    plt.show()

    # Find the best max_depth
    best_max_depth = max_depths[np.argmax(mean_cv_scores)]
    print(f"Best Max Depth: {best_max_depth} with Mean CV Accuracy: {round(max(mean_cv_scores) * 100, 2)}%")

    # Train and evaluate Random Forest with the best max_depth
    rdf_model_best = RandomForestClassifier(n_estimators=random_forest_estimator_num,
                                            max_depth=best_max_depth,
                                            criterion=criterion,
                                            random_state=42)
    rdf_model_best.fit(X, Y)
    scores_cv = cross_val_score(rdf_model_best, X, Y, cv=kf)
    mean_cv = scores_cv.mean()
    std_cv = scores_cv.std()

    # Display Cross-validation results
    print(f"Random Forest (Best Max Depth = {best_max_depth}) Cross-validation scores: {round(mean_cv * 100, 2)}%")
    print(f"Random Forest (Best Max Depth = {best_max_depth}) Standard deviation: {round(std_cv * 100, 2)}%")
    
    # Generate classification report
    y_pred = cross_val_predict(rdf_model_best, X, Y, cv=kf)
    report = classification_report(Y, y_pred, output_dict=True)
    cm = confusion_matrix(Y, y_pred)
    
    rdf_result = [{
        'Classifier': 'Random Forest',
        'Model Definition': rdf_model_best,
        'Class 0 - Precision': report['0']['precision'],
        'Class 0 - Recall': report['0']['recall'],
        'Class 0 - F1-Score': report['0']['f1-score'],
        'Class 1 - Precision': report['1']['precision'],
        'Class 1 - Recall': report['1']['recall'],
        'Class 1 - F1-Score': report['1']['f1-score'],
        'Average - Precision': report['macro avg']['precision'],
        'Average - Recall': report['macro avg']['recall'],
        'Average - F1-Score': report['macro avg']['f1-score'],
        'Accuracy': report['accuracy'],
        'Confusion Matrix': cm
    }]
    
    rdf_result_df = pd.DataFrame(rdf_result)
    
    display(Markdown("### Classification Report of Random Forest (Best Max Depth)"))
    display(rdf_result_df)
    
    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix - Random Forest (Best Max Depth)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    print()  # Add Blank Line
    display(Markdown("<span style='color: green; font-weight: bold;'>Random Forest Model Run Complete</span>"))
    print()  # Add Blank Line

    return rdf_model_best, mean_cv, std_cv

# Call the tune_random_forest function with the desired parameters and your DataFrame
rdf_model_best, mean_cv_best, std_cv_best = random_forest(df_norm)

### Export Random Forest as pickle

In [ ]:
savepath = r'Export Model'
rdf_filename_model = os.path.join(savepath, 'Model_RDF.sav')
pickle.dump(rdf_model_best, open(rdf_filename_model, 'wb'))